# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import math
import string
import mltools #user_defined package

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [3]:
products.head()
products = products.fillna({'review':''})

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products.iloc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation)

products['review_clean'] = products['review'].apply(remove_punctuation)
#products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
#products.loc[269,'word_count']

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_index = pd.read_json('module-2-assignment-train-idx.json',typ='series')
test_index = pd.read_json('module-2-assignment-test-idx.json',typ='series')

In [10]:
train_data = products.iloc[train_index]
test_data = products.iloc[test_index]
print len(train_data)
print len(test_data)

133416
33336


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [12]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(n_jobs=-1,verbose=1,
    #solver='lbfgs',max_iter=1000,
                                    )
sentiment_model.fit(train_matrix,train_data['sentiment'])

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

In [13]:
sentiment_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [14]:
feature_names = ['intercept']+vectorizer.vocabulary_.keys()
feature_values = np.hstack((sentiment_model.intercept_ ,sentiment_model.coef_[0]))

In [15]:
weights = mltools.get_name_value(feature_names,feature_values)
weights.columns

Index([u'abs_', u'name', u'value'], dtype='object')

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [16]:
num_positive_weights = len(weights[weights['value']>=0])
num_negative_weights = len(weights[weights['value']<0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 86827 
Number of negative weights: 34886 


68419  
53294  
**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [17]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

59    5
71    2
91    1
Name: rating, dtype: int64


,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [18]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [19]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [20]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60223347  -3.17035343 -10.42246382]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [21]:
yhat = [1 if score >0 else -1 for score in scores]
yhat

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [22]:
print "Class predictions according to Scikit Learn:" 
print sentiment_model.predict(sample_test_matrix)

Class predictions according to Scikit Learn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [23]:
P = [1/(1+math.exp(-score))  for score in scores]
P

[0.9963239493421193, 0.04029674506663104, 2.9755589004531535e-05]

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [24]:
print "Class predictions according to scikit learn:" 
print sentiment_model.predict_proba(sample_test_matrix)

Class predictions according to scikit learn:
[[  3.67605066e-03   9.96323949e-01]
 [  9.59703255e-01   4.02967451e-02]
 [  9.99970244e-01   2.97555890e-05]]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [25]:
test_data['probability'] = sentiment_model.predict_proba(test_matrix)[:,1]

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [26]:
test_data[test_data['probability']==1.0]['name']

22586        Britax Decathlon Convertible Car Seat, Tiffany
50315            P'Kolino Silly Soft Seating in Tias, Green
52631     Evenflo X Sport Plus Convenience Stroller - Ch...
66059          Evenflo 6 Pack Classic Glass Bottle, 4-Ounce
80155     Simple Wishes Hands-Free Breastpump Bra, Pink,...
87017       Baby Einstein Around The World Discovery Center
97325     Freemie Hands-Free Concealable Breast Pump Col...
100166    Infantino Wrap and Tie Baby Carrier, Black Blu...
114796    Fisher-Price Cradle 'N Swing,  My Little Snuga...
119182    Roan Rocco Classic Pram Stroller 2-in-1 with B...
133651                    Britax 2012 B-Agile Stroller, Red
137034           Graco Pack 'n Play Element Playard - Flint
140816           Diono RadianRXT Convertible Car Seat, Plum
147949    Baby Jogger City Mini GT Single Stroller, Shad...
168081    Buttons Cloth Diaper Cover - One Size - 8 Colo...
168697    Graco FastAction Fold Jogger Click Connect Str...
180646        Mamas &amp; Papas 2014 Urb

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [27]:
ans = test_data.sort_values('probability',ascending=False)['name']

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [28]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct = sum(predictions==true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct/len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [29]:
#from sklearn.metrics import accuracy_score
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.93229541636669067

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [30]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [31]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [32]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

Let's see what the first example of the dataset looks like:

In [33]:
train_data.iloc[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [34]:
#print train_data[0]['word_count']

Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [35]:
#print train_data[0]['word_count_subset']

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [36]:
simple_model = LogisticRegression(n_jobs=-1,verbose=1)
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])
simple_model

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [37]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.86936045116390692

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [38]:
simple_model_coef_table = mltools.get_name_value(significant_words,simple_model.coef_.flatten())

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [39]:
simple_model_coef_table.sort_values('value',ascending=False)

,abs_,name,value
6,1.673074,loves,1.673074
5,1.509812,perfect,1.509812
0,1.363690,love,1.363690
2,1.192538,easy,1.192538
1,0.944000,great,0.944000
4,0.520186,little,0.520186
7,0.503760,well,0.503760
8,0.190909,able,0.190909
3,0.085513,old,0.085513
9,0.058855,car,0.058855


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [40]:
simple_model_coef_table[simple_model_coef_table.value>0].shape

(10, 3)

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [41]:
positive_significant_words = simple_model_coef_table[simple_model_coef_table.value>0]['name']

In [42]:
for word in positive_significant_words:
    if word  in weights[weights.value>0]['name'].values:
        print 'true'

true
true
true
true
true
true
true


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [43]:
get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])

0.96851951789890267

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [44]:
get_classification_accuracy(simple_model, train_matrix_word_subset, train_data['sentiment'])

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [45]:
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.93229541636669067

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [46]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.86936045116390692

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [47]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [48]:
accuracy_baseline = (test_data['sentiment'] == +1).sum()/len(test_data)
print accuracy_baseline

0.842782577394


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?